In [ ]:
!pip install transformers datasets!pip install peft!pip install trl!pip install torch

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArgumentsfrom peft import LoraConfig, get_peft_model, prepare_model_for_kbit_trainingfrom datasets import load_datasetfrom trl import SFTTrainerimport torch

In [ ]:
model_name = "microsoft/DialoGPT-medium"tokenizer = AutoTokenizer.from_pretrained(model_name)tokenizer.pad_token = tokenizer.eos_token

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="auto")dataset = load_dataset("imdb")

In [ ]:
def tokenize_function(examples):    return tokenizer(        examples["text"],        padding="max_length",        truncation=True,        max_length=512,    )

In [ ]:
tokenized_dataset = dataset.map(tokenize_function)

In [ ]:
model = prepare_model_for_kbit_training(model)

In [ ]:
lora_config = LoraConfig(    r=16,    lora_alpha=32,    lora_dropout=0.05,    bias="none",    task_type="CAUSAL_LM",    target_modules=["c_attn", "c_proj"])

In [ ]:
model = get_peft_model(model, lora_config)model.print_trainable_parameters()

In [ ]:
training_args = TrainingArguments(    output_dir="./lora-dialogpt",    num_train_epochs=1,    per_device_train_batch_size=4,    gradient_accumulation_steps=4,    warmup_steps=100,    max_steps=1000,    learning_rate=2e-4,    fp16=True,    logging_steps=25,    save_strategy="steps",    save_steps=500,)

In [ ]:
trainer = SFTTrainer(    model=model,    train_dataset=tokenized_dataset["train"],    args=training_args,    processing_class=tokenizer,)

In [ ]:
trainer.train()

In [ ]:
model.save_pretrained("./lora-adapter")eval_results = trainer.evaluate()print(f"Evaluation results: {eval_results}")